# Задание по лекциям 1.

- Не нужно использовать эффективные вычислительные алгоритмы. Нужно использовать те алгоритмы, что были представлены на лекции или практических занятиях. 
- Остальное как обычно: за "похожие" решения всем задействованным 0 баллов; если используете решение из открытого источника — обязательно вставьте ссылку; не удаляйте формулировки; не выкладывайте в открытый источник.
- Можно использовать `numpy.array` для матриц и матричной арифметики и `numpy.linalg` для подсчёта ранга и определителя, для вычисления обратной матрицы, решения СЛУ и т.п. То есть то, что вы уже реализовывали в прошлом семестре, ещё раз реализовывать необязательно. Более того, можно использовать в любом из *заданий по лекциям* функции, реализованные ранее в других *заданиях по лекциям*. Если возникнут сомнения, можно ли использовать ту или иную функцию — лучше сразу поинтересуйтесь у меня.

$\mathbb{R}^n$ — вещественнозначное пространство вектор-**столбцов**.

**(1 балл) Задание 1.** Реализовать функцию, принимающую на вход два набора 
$$
u = (u_1, \ldots, u_n),\ v = (v_1,\ldots,v_n)
$$
координат векторов (в стандартном базисе) из пространства $\mathbb{R}^n$, и выдающую 
- матрицу перехода от базиса $u$ к базису $v$, если оба этих набора являются базисами;
- None, иначе (альтернативный вариант — кидать исключение).

In [16]:
import numpy as np

def transform_mx(u, v):
    if np.linalg.det(u) and np.linalg.det(v):
        return np.linalg.inv(u) @ v

    
u = np.array([[1, 2, 3], 
              [2, 3, 7], 
              [1, 3, 1]])

v = np.array([[3, 1, 4], 
              [5, 2, 1], 
              [1, 1, -6]])

t = transform_mx(u, v)
print(np.allclose(u @ t, v)) # Проверка

u = np.array([[1, 2, 3], [2, 4, 6], [1, 3, 1]])
print(transform_mx(u, v)) # Набор векторов U не является базисом

a = np.array([[8,-6,7],[-16,7,-13],[9,-3,7]])
b = np.array([[1,-2,1],[3,-1,2],[2,1,2]])


True
None


**(1 балл) Задание 2.** Реализовать функцию, принимающую на вход матрицу $A$ линейного оператора $\varphi: \mathbb{R}^n \to \mathbb{R}^n$ (в стандартном базисе) и набор координат $u = (u_1,\ldots,u_n)$ вектор-столбцов, и выдающую 
- матрицу оператора $\varphi$ в базисе $u$, если $u$ является базисом;
- None, иначе (альтернативный вариант — кидать исключение).

In [9]:
def mx_operator(A, u):
    if(np.linalg.det(u)):
        return np.linalg.inv(u) @ A @ u

A = np.array([[1, 2, 0, 1], 
              [3, 0, -1, 2], 
              [2, 5, 3, 1], 
              [1, 2, 1, 3]]) # Проскуряков 1452 a)

u = np.array([[1, 0, 0, 0], 
              [0, 0, 1, 0], 
              [0, 1, 0, 0], 
              [0, 0, 0, 1]])

print(mx_operator(A, u))

[[ 1.  0.  2.  1.]
 [ 2.  3.  5.  1.]
 [ 3. -1.  0.  2.]
 [ 1.  1.  2.  3.]]


**(1.5 балла) Задание 3.** Реализовать функции, которые по данной матрице $A$ линейного оператора $\varphi: \mathbb{R}^n \to \mathbb{R}^n$ в стандартном базисе, выдадут:

- базис образа $\varphi$ (0.5 балла)
- базис ядра $\varphi$ (0.75 балла)
- размерности ядра и образа $\varphi$ (0.25 балла)

Использую два алгоритма(стр. 7) из этого источника http://kadm.kmath.ru/files/linalg16.pdf
А также тест с последнего слайда

In [12]:
import sympy as sp

def basis_im(A):
    vs = np.array(sp.Matrix(A.T).echelon_form().T)
    vs = vs[:, np.any(vs != 0, axis=0)]
    return vs
        

def basis_ker(A):
    space = sp.Matrix(A).nullspace()
    if space:
        basis = sp.Matrix()
        for v in space:
            basis = basis.col_insert(sp.shape(v)[1], v)
        return np.array(basis)
    
    return np.zeros((A.shape[0], 1))

def dimension_ker_im(A):
    ker = basis_ker(A)
    im = basis_ker(A)
    return ker.shape[1], im.shape[1]

A = np.array([[2, 0, 1, -3], 
              [1, 0, 3, -4], 
              [-1, 0, 2, -1], 
              [1, 0, 1, -2]])
print(basis_im(A))
print(basis_ker(A))
print(dimension_ker_im(A))

[[2 0]
 [1 5]
 [-1 5]
 [1 1]]
[[0 1]
 [1 0]
 [0 1]
 [0 1]]
(2, 2)


**(1 балл) Задание 4.** Реализовать функцию, принимающую на вход матрицу $A$ линейного оператора $\varphi: \mathbb{R}^n \to \mathbb{R}^n$ (в стандартном базисе) и набор координат $u = (u_1,\ldots,u_k),\ k\le n$ вектор-столбцов, и выдающую 
- True, если линейная оболочка векторов $(u_1,\ldots, u_k)$ является $\varphi$-инвариантным подпространством.
- False, иначе.

In [17]:
def is_invariant(A, U):
    k_col = U.shape[1]
    for i in range(k_col):
        B = A @ U[:, i]
        rank_U = np.linalg.matrix_rank(U)
        UB = np.c_[U, B]
        rank_UB = np.linalg.matrix_rank(UB)
        if rank_U != rank_UB:
            return False
        
    return True

A = np.array([[1, 0, 0], 
              [0, 0, 0], 
              [0, 0, 0]])

U = np.array([[1, 2, 1], 
              [2, 4, 0], 
              [3, 6, 0]])

print(is_invariant(A, U))
# any v from L(U), Av in L(U), bcoz v = (a, b, c).T, Av = (a, 0, 0).T and (1, 0, 0) in U 

A = np.array([[1, 0, 0], 
              [1, 0, 0], 
              [1, 0, 0]])

U = np.array([[1, 1], 
              [0, 2], 
              [0, 3]])
print(is_invariant(A, U))
# exist v = (1, 2, 3).T from L(U), Av not in L(U), bcoz Av = (1, 1, 1).T and does not exist x1, x2: x1 * u_1 + x2 * u_2 = (a, a, a).T  

True
False
